In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import svm, datasets
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import display
import pandas as pd
import nltk
import ssl
from sklearn.svm import LinearSVC
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
import regex
from sklearn.compose import ColumnTransformer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vaibhavidharashivkar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("../data/interim/ecommerce_data-cleaned-0.2.2.csv", index_col=0, encoding='utf-8')
display(df)

,brand,name,description,category_raw,price_raw,discount_raw,price_range
0,La Costeï¿½ï¿½a,"La Costena Chipotle Peppers, 7 OZ (Pack of 12)",We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",31.93,31.93,"(30, 35]"
1,Equate,Equate Triamcinolone Acetonide Nasal Allergy S...,We aim to show you accurate product informati...,Health | Equate | Equate Allergy | Equate Sinu...,10.48,10.48,"(10, 20]"
2,AduroSmart ERIA,AduroSmart ERIA Soft White Smart A19 Light Bul...,We aim to show you accurate product informati...,Electronics | Smart Home | Smart Energy and Li...,10.99,10.99,"(10, 20]"
3,lowrider,"24"" Classic Adjustable Balloon Fender Set Chro...",We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Accessories |...,38.59,38.59,"(35, 40]"
4,Anself,Elephant Shape Silicone Drinkware Portable Sil...,We aim to show you accurate product informati...,Baby | Feeding | Sippy Cups: Alternatives to P...,5.81,5.81,"(0, 10]"
...,...,...,...,...,...,...,...
29994,NineChef,Sheng Xiang Zhen (ShengXiangZhen) Snack + OneN...,We aim to show you accurate product informati...,"Food | Snacks, Cookies & Chips | Chips & Crisp...",45.99,45.99,"(45, 50]"
29996,Shock Sox,Shock Sox Fork Seal Guards 29-36mm Fork Tube 4...,We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Components | ...,33.25,33.25,"(30, 35]"
29997,Princes,Princes Gooseberries 300g,We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",8.88,8.88,"(0, 10]"
29998,Create Ion,Create Ion Grace 3/4 Inches Straight Hair Iron...,We aim to show you accurate product informati...,Beauty | Hair Care | Hair Styling Tools | Flat...,50.00,24.50,"(45, 50]"


In [3]:
def get_range_label(price):
    value = np.round(price, decimals=1)
    if value <= 10:
        return 0
    elif 10 < value <= 20:
        return 1
    elif 20 < value <= 25:
        return 2
    elif 25 < value <= 30:
        return 3
    elif 30 < value <= 35:
        return 4
    elif 35 < value <= 40:
        return 5
    elif 40 < value <= 45:
        return 6
    elif 45 < value <= 50:
        return 7
    elif 50 < value <= 100:
        return 8
    else:
        return 9

df['labels'] = df['price_raw'].apply(lambda x: get_range_label(x))

In [4]:
stemmer = PorterStemmer()
words = stopwords.words("english")
cleaned_text = lambda x: " ".join([stemmer.stem(i) for i in regex.sub("[^a-zA-Z0-9]", " ", x).split() if i not in words]).lower()
df['cleaned_name'] = df.name.apply(cleaned_text)
df['cleaned_brand'] = df.brand.fillna("").apply(cleaned_text)
df['cleaned_description'] = df.description.fillna("").apply(cleaned_text)
df['cleaned_category_raw'] = df.category_raw.fillna("").apply(cleaned_text)
display(df)

,brand,name,description,category_raw,price_raw,discount_raw,price_range,labels,cleaned_name,cleaned_brand,cleaned_description,cleaned_category_raw
0,La Costeï¿½ï¿½a,"La Costena Chipotle Peppers, 7 OZ (Pack of 12)",We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",31.93,31.93,"(30, 35]",4,la costena chipotl pepper 7 oz pack 12,la cost,we aim show accur product inform manufactur su...,food meal solut grain pasta can good can veget
1,Equate,Equate Triamcinolone Acetonide Nasal Allergy S...,We aim to show you accurate product informati...,Health | Equate | Equate Allergy | Equate Sinu...,10.48,10.48,"(10, 20]",1,equat triamcinolon acetonid nasal allergi spra...,equat,we aim show accur product inform manufactur su...,health equat equat allergi equat sinu congest ...
2,AduroSmart ERIA,AduroSmart ERIA Soft White Smart A19 Light Bul...,We aim to show you accurate product informati...,Electronics | Smart Home | Smart Energy and Li...,10.99,10.99,"(10, 20]",1,adurosmart eria soft white smart a19 light bul...,adurosmart eria,we aim show accur product inform manufactur su...,electron smart home smart energi light smart l...
3,lowrider,"24"" Classic Adjustable Balloon Fender Set Chro...",We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Accessories |...,38.59,38.59,"(35, 40]",5,24 classic adjust balloon fender set chrome bi...,lowrid,we aim show accur product inform manufactur su...,sport outdoor bike bike accessori bike fender
4,Anself,Elephant Shape Silicone Drinkware Portable Sil...,We aim to show you accurate product informati...,Baby | Feeding | Sippy Cups: Alternatives to P...,5.81,5.81,"(0, 10]",0,eleph shape silicon drinkwar portabl silicon c...,anself,we aim show accur product inform manufactur su...,babi feed sippi cup altern plastic
...,...,...,...,...,...,...,...,...,...,...,...,...
29994,NineChef,Sheng Xiang Zhen (ShengXiangZhen) Snack + OneN...,We aim to show you accurate product informati...,"Food | Snacks, Cookies & Chips | Chips & Crisp...",45.99,45.99,"(45, 50]",7,sheng xiang zhen shengxiangzhen snack onenin c...,ninechef,we aim show accur product inform manufactur su...,food snack cooki chip chip crisp chip crisp
29996,Shock Sox,Shock Sox Fork Seal Guards 29-36mm Fork Tube 4...,We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Components | ...,33.25,33.25,"(30, 35]",4,shock sox fork seal guard 29 36mm fork tube 4 ...,shock sox,we aim show accur product inform manufactur su...,sport outdoor bike bike compon bike fork
29997,Princes,Princes Gooseberries 300g,We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",8.88,8.88,"(0, 10]",0,princ gooseberri 300g,princ,we aim show accur product inform manufactur su...,food meal solut grain pasta can good can fruit
29998,Create Ion,Create Ion Grace 3/4 Inches Straight Hair Iron...,We aim to show you accurate product informati...,Beauty | Hair Care | Hair Styling Tools | Flat...,50.00,24.50,"(45, 50]",7,creat ion grace 3 4 inch straight hair iron ci...,creat ion,we aim show accur product inform manufactur su...,beauti hair care hair style tool flat iron hai...


In [5]:
vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29604 entries, 0 to 29999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   brand                 29045 non-null  object 
 1   name                  29604 non-null  object 
 2   description           29552 non-null  object 
 3   category_raw          29588 non-null  object 
 4   price_raw             29604 non-null  float64
 5   discount_raw          29604 non-null  float64
 6   price_range           29604 non-null  object 
 7   labels                29604 non-null  int64  
 8   cleaned_name          29604 non-null  object 
 9   cleaned_brand         29604 non-null  object 
 10  cleaned_description   29604 non-null  object 
 11  cleaned_category_raw  29604 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 2.9+ MB
None


In [13]:
X = [df['cleaned_name'], df['cleaned_description'], df['cleaned_brand'], df['cleaned_category_raw']]

for i in X:
    X_train, X_test, y_train, y_test = train_test_split(i, df['labels'], test_size=0.25)
    
    pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo'))])

    model = pipeline.fit(X_train, y_train)

    ytest = np.array(y_test)

    print(classification_report(ytest, model.predict(X_test)))
    print(confusion_matrix(ytest, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.43      0.34      0.38      1648
           1       0.30      0.66      0.41      1922
           2       0.12      0.02      0.04       580
           3       0.26      0.03      0.06       504
           4       0.16      0.02      0.04       323
           5       0.05      0.01      0.01       327
           6       0.06      0.01      0.02       220
           7       0.11      0.02      0.03       222
           8       0.29      0.23      0.25       837
           9       0.41      0.41      0.41       818

    accuracy                           0.32      7401
   macro avg       0.22      0.17      0.17      7401
weighted avg       0.29      0.32      0.28      7401

[[ 563  972   12   10    2    4    2    4   35   44]
 [ 398 1262   37   14    3    9   14    6   88   91]
 [  81  394   14    4    5    5    5    1   42   29]
 [  67  319   11   17    8    4    7    0   31   40]
 [  35  181    6    4    7   

In [9]:
# linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train)
# rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(X_train, y_train)
# poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(X_train, y_train)
# sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(X_train, y_train)

In [14]:
from sklearn.compose import ColumnTransformer

X = df
X_train, X_test, y_train, y_test = train_test_split(X, df['labels'], test_size=0.25)
column_trans = ColumnTransformer([('name', vectorizer, 'cleaned_name'),('description', vectorizer, 'cleaned_description'), 
                                  ('brand', vectorizer, 'cleaned_brand'), ('category_raw', vectorizer, 'cleaned_category_raw')],
                                remainder='drop', verbose_feature_names_out=False)

pipeline = Pipeline([('vect', column_trans),
                     ('chi',  SelectKBest(chi2, k=7000)),
                     ('clf', svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo'))])

model = pipeline.fit(X_train, y_train)

ytest = np.array(y_test)

print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))



              precision    recall  f1-score   support

           0       0.53      0.53      0.53      1635
           1       0.36      0.69      0.47      1923
           2       0.29      0.07      0.11       586
           3       0.36      0.08      0.13       521
           4       0.45      0.07      0.12       315
           5       0.62      0.08      0.15       312
           6       0.79      0.11      0.19       211
           7       0.41      0.03      0.06       219
           8       0.37      0.38      0.38       857
           9       0.60      0.61      0.60       822

    accuracy                           0.43      7401
   macro avg       0.48      0.27      0.27      7401
weighted avg       0.45      0.43      0.39      7401

[[ 864  707   10    3    0    0    0    0   33   18]
 [ 417 1329   19    9    7    5    0    1   92   44]
 [  84  375   39   19    3    0    0    1   47   18]
 [  66  308   24   43    2    2    1    0   59   16]
 [  34  166    9    9   21   